# HyperParameter Tuning

### `keras.wrappers.scikit_learn`

Example adapted from: [https://github.com/fchollet/keras/blob/master/examples/mnist_sklearn_wrapper.py]()

## Problem: 
Builds simple CNN models on MNIST and uses sklearn's GridSearchCV to find best model

In [1]:
import numpy as np
np.random.seed(1337)  # for reproducibility

In [2]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K

Using TensorFlow backend.


In [3]:
from sklearn.model_selection import GridSearchCV

# Data Preparation

In [4]:
nb_classes = 10

# input image dimensions
img_rows, img_cols = 28, 28

In [5]:
# load training data and do basic data normalization
(X_train, y_train), (X_test, y_test) = mnist.load_data()

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [6]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

## Build Model

In [7]:
def make_model(dense_layer_sizes, filters, kernel_size, pool_size):
    '''Creates model comprised of 2 convolutional layers followed by dense layers

    dense_layer_sizes: List of layer sizes. This list has one number for each layer
    nb_filters: Number of convolutional filters in each convolutional layer
    nb_conv: Convolutional kernel size
    nb_pool: Size of pooling area for max pooling
    '''

    model = Sequential()

    model.add(Conv2D(filters, (kernel_size, kernel_size),
                     padding='valid', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(filters, (kernel_size, kernel_size)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    for layer_size in dense_layer_sizes:
        model.add(Dense(layer_size))
        model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    return model

In [8]:
dense_size_candidates = [[32], [64], [32, 32], [64, 64]]
my_classifier = KerasClassifier(make_model, batch_size=32)

## GridSearch HyperParameters

In [ ]:
validator = GridSearchCV(my_classifier,
                         param_grid={'dense_layer_sizes': dense_size_candidates,
                                     # nb_epoch is avail for tuning even when not
                                     # an argument to model building function
                                     'epochs': [3, 6],
                                     'filters': [8],
                                     'kernel_size': [3],
                                     'pool_size': [2]},
                         scoring='neg_log_loss',
                         n_jobs=1,
                         cv=5)
validator.fit(X_train, y_train)

Epoch 1/3
48000/48000 [==============================] - 6s 127us/step - loss: 0.5854 - acc: 0.8083
Epoch 2/3
48000/48000 [==============================] - 5s 94us/step - loss: 0.3402 - acc: 0.8936
Epoch 3/3
48000/48000 [==============================] - 5s 95us/step - loss: 0.2808 - acc: 0.9136
Epoch 1/3
48000/48000 [==============================] - 5s 106us/step - loss: 0.5618 - acc: 0.8134
Epoch 2/3
48000/48000 [==============================] - 5s 94us/step - loss: 0.3373 - acc: 0.8947
Epoch 3/3
48000/48000 [==============================] - 5s 94us/step - loss: 0.2880 - acc: 0.9092
Epoch 1/3
48000/48000 [==============================] - 5s 108us/step - loss: 0.6220 - acc: 0.7976
Epoch 2/3
48000/48000 [==============================] - 5s 96us/step - loss: 0.3572 - acc: 0.8865
Epoch 3/3
48000/48000 [==============================] - 5s 95us/step - loss: 0.2999 - acc: 0.9048
Epoch 1/3
48000/48000 [==============================] - 5s 109us/step - loss: 0.5490 - acc: 0.8257
Epoch 

48000/48000 [==============================] - 5s 100us/step - loss: 0.1396 - acc: 0.9600
Epoch 6/6
48000/48000 [==============================] - 5s 101us/step - loss: 0.1272 - acc: 0.9639
Epoch 1/6
48000/48000 [==============================] - 6s 127us/step - loss: 0.4168 - acc: 0.8715
Epoch 2/6
48000/48000 [==============================] - 5s 102us/step - loss: 0.2014 - acc: 0.9397
Epoch 3/6
48000/48000 [==============================] - 5s 102us/step - loss: 0.1642 - acc: 0.9512
Epoch 4/6
48000/48000 [==============================] - 5s 101us/step - loss: 0.1487 - acc: 0.9568
Epoch 5/6
48000/48000 [==============================] - 5s 101us/step - loss: 0.1331 - acc: 0.9611
Epoch 6/6
48000/48000 [==============================] - 5s 101us/step - loss: 0.1230 - acc: 0.9641
Epoch 1/3
48000/48000 [==============================] - 6s 132us/step - loss: 0.5770 - acc: 0.8110
Epoch 2/3
48000/48000 [==============================] - 5s 103us/step - loss: 0.2869 - acc: 0.9122
Epoch 3/3


48000/48000 [==============================] - 5s 114us/step - loss: 0.1159 - acc: 0.9682
Epoch 4/6
48000/48000 [==============================] - 5s 114us/step - loss: 0.0970 - acc: 0.9746
Epoch 5/6
48000/48000 [==============================] - 5s 114us/step - loss: 0.0893 - acc: 0.9772
Epoch 6/6
48000/48000 [==============================] - 5s 114us/step - loss: 0.0814 - acc: 0.9787
Epoch 1/6
48000/48000 [==============================] - 8s 156us/step - loss: 0.3518 - acc: 0.8932
Epoch 2/6
48000/48000 [==============================] - 5s 114us/step - loss: 0.1469 - acc: 0.9609
Epoch 3/6
48000/48000 [==============================] - 6s 115us/step - loss: 0.1124 - acc: 0.9699
Epoch 4/6
48000/48000 [==============================] - 6s 115us/step - loss: 0.0967 - acc: 0.9738
Epoch 5/6
48000/48000 [==============================] - 5s 115us/step - loss: 0.0846 - acc: 0.9774
Epoch 6/6
48000/48000 [==============================] - 5s 115us/step - loss: 0.0766 - acc: 0.9791
Epoch 1/6


In [10]:
print('The parameters of the best model are: ')
print(validator.best_params_)

# validator.best_estimator_ returns sklearn-wrapped version of best model.
# validator.best_estimator_.model returns the (unwrapped) keras model
best_model = validator.best_estimator_.model
metric_names = best_model.metrics_names
metric_values = best_model.evaluate(X_test, y_test)
for metric, value in zip(metric_names, metric_values):
    print(metric, ': ', value)

The parameters of the best model are: 
{'dense_layer_sizes': [64, 64], 'epochs': 6, 'filters': 8, 'kernel_size': 3, 'pool_size': 2}
10000/10000 [==============================] - 2s 249us/step
loss :  0.04074383126419634
acc :  0.989


---

# There's more:

The `GridSearchCV` model in scikit-learn performs a complete search, considering **all** the possible combinations of Hyper-parameters we want to optimise.

If we want to apply for an optmised and bounded search in the hyper-parameter space, I strongly suggest to take a look at:

* `Keras + hyperopt == hyperas`: [http://maxpumperla.github.io/hyperas/](http://maxpumperla.github.io/hyperas/)